In [1]:
#패키지 로드 및 플래그값 정의  
import data_utils
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import os
from tensorflow.models.rnn.translate import seq2seq_model
import sys
from pprint import pprint
tf.app.flags.DEFINE_float("learning_rate", 0.5, "Learning rate.")
tf.app.flags.DEFINE_float("max_gradient_norm", 5.0, "Clip gradients to this norm.")
tf.app.flags.DEFINE_float("learning_rate_decay_factor", 0.99, "Learning rate decays by this much.")
tf.app.flags.DEFINE_integer("batch_size", 64, "Batch size to use during training.")
tf.app.flags.DEFINE_integer("size", 128, "Size of each model layer.")
tf.app.flags.DEFINE_integer("num_layers", 2, "Number of layers in the model.")
tf.app.flags.DEFINE_integer("en_vocab_size", 40, "English vocabulary size.")
tf.app.flags.DEFINE_integer("fr_vocab_size", 1000, "French vocabulary size.")
tf.app.flags.DEFINE_string("data_dir", "data", "Data directory")
tf.app.flags.DEFINE_string("train_dir", "train", "Training directory.")
tf.app.flags.DEFINE_integer("max_train_data_size", 0, "Limit on the size of training data (0: no limit).")
tf.app.flags.DEFINE_integer("steps_per_checkpoint", 100, "How many training steps to do per checkpoint.")
tf.app.flags.DEFINE_boolean("decode", False, "Set to True for interactive decoding.")
tf.app.flags.DEFINE_boolean("self_test", False, "Run a self-test if this is set to True.")
tf.app.flags.DEFINE_boolean("use_lstm", True, "Set to True for using lstm instead gru cell.")
FLAGS = tf.app.flags.FLAGS

/Users/nhnent/miniconda3/envs/trans2/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/Users/nhnent/miniconda3/envs/trans2/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/Users/nhnent/miniconda3/envs/trans2/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/Users/nhnent/miniconda3/envs/trans2/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/Users/nhnent/miniconda

In [2]:
#학습파일,테스트파일 불러오기
print("Preparing WMT data in %s" % FLAGS.data_dir)
en_train, fr_train, en_dev, fr_dev, en_idx, fr_idx = data_utils.prepare_wmt_data(FLAGS.data_dir, FLAGS.en_vocab_size, FLAGS.fr_vocab_size)
en_train, fr_train, en_dev, fr_dev, en_idx, fr_idx

Preparing WMT data in data


('data/giga-fren.release2.ids40.en',
 'data/giga-fren.release2.ids1000.fr',
 'data/newstest2013.ids40.en',
 'data/newstest2013.ids1000.fr',
 'data/vocab40.en',
 'data/vocab1000.fr')

In [3]:
## 학습모델 폴더가 없을 경우 만들어주기
if not os.path.exists(FLAGS.train_dir):
    os.mkdir(FLAGS.train_dir)

In [4]:
#모델을 정의하는 부분
#def create_model(session, forward_only):
'''Create translation model and initialize or load parameters in session.'''
forward_only = False
_buckets = [(7, 5), (10, 7), (17, 10), (37, 20)] #버켓사이즈
model = seq2seq_model.Seq2SeqModel(
    FLAGS.en_vocab_size
    , FLAGS.fr_vocab_size
    , _buckets
    , FLAGS.size
    , FLAGS.num_layers
    , FLAGS.max_gradient_norm
    , FLAGS.batch_size
    , FLAGS.learning_rate
    , FLAGS.learning_rate_decay_factor
    , forward_only=forward_only
    , use_lstm=FLAGS.use_lstm
)
model

In [5]:
#train_dir에 잇는 체크포인트 모두 가져오기
ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
ckpt

In [6]:
#체크포인트가 있을 경우 모델 재로딩 하고 그렇지 않을 경우 변수 초기화
sess = tf.InteractiveSession()
if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
    print('Model already exists')
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("Created model with fresh parameters.")
    sess.run(tf.initialize_all_variables())

Created model with fresh parameters.


In [7]:
#Read data into buckets and compute their sizes.
print("Reading development and training data (limit: %d)." % FLAGS.max_train_data_size)

Reading development and training data (limit: 0).


In [8]:
#학습/테스트 데이터를 읽어들이는 함수
def read_data(source_path, target_path, max_size=None):
    """Read data from source and target files and put into buckets.
    Args:
    source_path: path to the files with token-ids for the source language.
    target_path: path to the file with token-ids for the target language;
      it must be aligned with the source file: n-th line contains the desired
      output for n-th line from the source_path.
    max_size: maximum number of lines to read, all other will be ignored;
      if 0 or None, data files will be read completely (no limit).
    Returns:
    data_set: a list of length len(_buckets); data_set[n] contains a list of
      (source, target) pairs read from the provided data files that fit
      into the n-th bucket, i.e., such that len(source) < _buckets[n][0] and
      len(target) < _buckets[n][1]; source and target are lists of token-ids.
    """
    data_set = [[] for _ in _buckets] #각 버켓마다 담을 그릇 준비
    diffs = [] #
    #소스 파일(영어)과 타겟 파일(한국어)을 모두 읽기
    with tf.gfile.GFile(source_path, mode="r") as source_file:
        with tf.gfile.GFile(target_path, mode="r") as target_file:
            #영어/한국어 한줄씩 읽기
            source, target = source_file.readline(), target_file.readline()
            counter = 0
            while source and target and (not max_size or counter < max_size):
                counter += 1
                #일정단위로 상태 출력
                if counter % 100000 == 0:
                    print("reading data line %d" % counter)
                    sys.stdout.flush()
                source_ids = [int(x) for x in source.split()]
                target_ids = [int(x) for x in target.split()]
                target_ids.append(data_utils.EOS_ID) #EOS 넣어주기
                for bucket_id, (source_size, target_size) in enumerate(_buckets):
                    if len(source_ids) < source_size and len(target_ids) < target_size:
                        data_set[bucket_id].append([source_ids, target_ids])
                        diffs.append(source_size - len(source_ids) + target_size - len(target_ids))
                        break
                source, target = source_file.readline(), target_file.readline()
    return data_set

In [9]:
#학습데이터와 테스트데이터 버켓에 담기 (개수 제한 없음: max_size=0)
import numpy as np
dev_set = read_data(en_dev, fr_dev)
train_set = read_data(en_train, fr_train, FLAGS.max_train_data_size) #FLAGS.max_train_data_size=0
len(dev_set), len(train_set)

(4, 4)

In [10]:
#각 버켓의 사이즈 표시
train_bucket_sizes = [len(train_set[b]) for b in xrange(len(_buckets))] #각 버켓의 사이즈
train_total_size = float(sum(train_bucket_sizes)) #전체 학습데이터 사이즈
train_bucket_sizes, train_total_size

([5983, 8376, 7279, 885], 22523.0)

In [11]:
#각 버켓의 분포를 4분위로 나누어서 보여주기
train_buckets_scale = [sum(train_bucket_sizes[:i + 1]) / train_total_size for i in xrange(len(train_bucket_sizes))]
train_buckets_scale

[0.26563956844114905, 0.6375260844470098, 0.96070683301514, 1.0]

### Training Model

In [12]:
# 학습과 관련된 초기값
step_time, loss = 0.0, 0.0
current_step = 0
previous_losses = []
best_eval_ppx = float('inf') #무한대

In [ ]:
#while문으로 학습시키기
import time
import math
sess = tf.InteractiveSession()
tf.initialize_all_variables().run()
current_step = 0
while True:
    # Choose a bucket according to data distribution. We pick a random number
    # in [0, 1] and use the corresponding interval in train_buckets_scale.
    random_number_01 = np.random.random_sample()
    bucket_id = min([i for i in xrange(len(train_buckets_scale)) if train_buckets_scale[i] > random_number_01])

    # Get a batch and make a step.
    start_time = time.time()
    encoder_inputs, decoder_inputs, target_weights = model.get_batch(train_set, bucket_id)
    _, step_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, False)
    step_time += (time.time() - start_time) / FLAGS.steps_per_checkpoint
    loss += step_loss / FLAGS.steps_per_checkpoint
    current_step += 1

    # Once in a while, we save checkpoint, print statistics, and run evals.
    # 체크포인트 step마다 모델 저장하기
    if current_step % FLAGS.steps_per_checkpoint == 0:
        print('current_step={}'.format(current_step))
        # Print statistics for the previous epoch.
        perplexity = math.exp(loss) if loss < 300 else float('inf') #float('inf'): 무한인수
        print("global step %d learning rate %.4f step-time %.2f perplexity %.2f" % (model.global_step.eval(), model.learning_rate.eval(), step_time, perplexity))

        # Decrease learning rate if no improvement was seen over last 3 times.
        if len(previous_losses) > 2 and loss > max(previous_losses[-3:]):
            sess.run(model.learning_rate_decay_op)
            previous_losses.append(loss)

        # Save checkpoint and zero timer and loss.
        checkpoint_path = os.path.join(FLAGS.train_dir, "translate.ckpt")
        model.saver.save(sess, checkpoint_path, global_step=model.global_step)
        step_time, loss = 0.0, 0.0

        # Run evals on development set and print their perplexity.
        eval_ppx_list = []
        for bucket_id in xrange(len(_buckets)):
            if len(dev_set[bucket_id]) == 0:
                print("  eval: empty bucket %d" % (bucket_id))
                continue
            encoder_inputs, decoder_inputs, target_weights = model.get_batch(dev_set, bucket_id)
            _, eval_loss, _ = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id, True)
            eval_ppx = math.exp(eval_loss) if eval_loss < 300 else float('inf')
            eval_ppx_list.append(eval_ppx)
            #print("  eval_ppx_list={}".format(eval_ppx_list))
            print("  eval: bucket %d perplexity %.2f" % (bucket_id, eval_ppx))
        sys.stdout.flush()
    
        mean_eval_ppx = np.mean(eval_ppx_list)
        if mean_eval_ppx < best_eval_ppx:
            best_eval_ppx = mean_eval_ppx
            print("BEST mean eval perplexity: %.3f" % best_eval_ppx)
    
    #make break for test
    if current_step == 100:
        print('break')
        #break

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x1443b8a90>> ignored


current_step=100
global step 100 learning rate 0.5000 step-time 0.35 perplexity 258.14
  eval: bucket 0 perplexity 54.56
  eval: bucket 1 perplexity 90.78
  eval: bucket 2 perplexity 145.56
  eval: bucket 3 perplexity 370.67
break
current_step=200
global step 200 learning rate 0.5000 step-time 0.31 perplexity 82.85
  eval: bucket 0 perplexity 78.07
  eval: bucket 1 perplexity 99.39
  eval: bucket 2 perplexity 131.71
  eval: bucket 3 perplexity 131.32
current_step=300
global step 300 learning rate 0.5000 step-time 0.33 perplexity 55.55
  eval: bucket 0 perplexity 29.21
  eval: bucket 1 perplexity 37.21
  eval: bucket 2 perplexity 54.61
  eval: bucket 3 perplexity 106.98
BEST mean eval perplexity: 57.000
current_step=400
global step 400 learning rate 0.5000 step-time 0.31 perplexity 36.52
  eval: bucket 0 perplexity 26.08
  eval: bucket 1 perplexity 31.75
  eval: bucket 2 perplexity 40.18
  eval: bucket 3 perplexity 66.40
BEST mean eval perplexity: 41.103
current_step=500
global step 500